In [1]:
from flask import Flask, request, jsonify, session
import fitz  # PyMuPDF for PDF extraction
from google import genai
from google.genai import types


In [2]:
# pip install PyMuPDF

In [3]:
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Needed for session management


In [ ]:
client = genai.Client(api_key="")
grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)
config = types.GenerateContentConfig(
    tools=[grounding_tool]
)


In [5]:

@app.route('/evaluate_resume', methods=['POST'])
def evaluate_resume():
    if 'resume' not in request.files or 'jd' not in request.form:
        return jsonify({'error': 'Missing resume or JD'}), 400
    resume_file = request.files['resume']
    jd_text = request.form['jd']


    doc = fitz.open(stream=resume_file.read(), filetype="pdf")
    resume_text = "\n".join(page.get_text() for page in doc)


    system_prompt = (
        "You are an expert HR analyst. Evaluate the following resume against the provided job description (JD). "
        "Score the resume out of 100 based on relevance, skills, experience, and overall fit. "
        "Provide detailed suggestions to improve the resume for this JD. "
        "Return a JSON with 'score' and 'suggestions'.\n"
        f"JD: {jd_text}\nResume: {resume_text}"
    )
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=system_prompt,
        config=config,
    )
    return jsonify({'result': response.text})


In [6]:

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get('message')
    new_chat = data.get('new_chat', False)
    if new_chat or 'chat_obj' not in session:
        chat = client.chats.create(model="gemini-2.5-flash")
        # Add system message as first message
        chat.send_message("You are an Interview Coach Assistant. Help users prepare for interviews, give feedback, ask follow-up questions, and provide tips.")
        session['chat_obj'] = chat
    else:
        chat = session['chat_obj']
    response = chat.send_message(user_message)
    print(response.text)

    return "Success"

In [ ]:
app.run(port=2020)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:2020
Press CTRL+C to quit
[2025-09-11 23:02:55,623] ERROR in app: Exception on /chat [POST]
Traceback (most recent call last):
  File "c:\Users\Rajesh Akaike\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Rajesh Akaike\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
           ~~~~~~~~~~~~~~~~~~~~~^^^^
  File "c:\Users\Rajesh Akaike\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 941, in finalize_request
    response = self.process_response(response)
  File "c:\Users\Rajesh Akaike\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1322, in process_response
    self.session_interface.save_session(self, ctx.session, response)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

Hello there! Great to hear from you.

To make sure I can help you as effectively as possible, could you please tell me a bit more about the interview you're preparing for?

Specifically, I'd love to know:

1.  **What is the job title or role you're interviewing for?**
2.  **Which company is it with (if you know)?**
3.  **What industry is it in?**
4.  **What kind of interview is it?** (e.g., first-round, behavioral, technical, final-round)
5.  **What specifically would you like to practice or focus on today?**

Once I have this, we can jump right into preparing!
